# 建立研究智能體

研究智能體的工作：收集資料、提出問題、整理資訊

## 1. 載入基本設定

In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()

def ask_ai(prompt, role="你是一個研究助手"):
    """與 AI 對話的基本函數"""
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": role},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

## 2. 創建專家角色

不同專家會從不同角度看問題：

In [ ]:
def create_expert(topic):
    """
    根據主題創建相關專家
    
    例如：寫「咖啡」會產生咖啡師、營養師等專家
    """
    prompt = f"""
    主題：{topic}
    
    請列出 3 個最適合討論這個主題的專家角色。
    每個專家用一行，格式：專家名稱 - 專業領域
    
    例如：
    歷史學家 - 研究歷史發展
    營養師 - 分析健康影響
    """
    
    experts_text = ask_ai(prompt)
    
    # 解析專家列表
    experts = []
    for line in experts_text.strip().split('\n'):
        if '-' in line:
            experts.append(line.strip())
    
    return experts

# 測試
topic = "人工智慧"
experts = create_expert(topic)
print(f"針對『{topic}』主題的專家：")
for expert in experts:
    print(f"  • {expert}")

## 3. 產生研究問題

每個專家會提出不同的問題：

In [ ]:
def generate_questions(topic, expert):
    """
    讓專家針對主題提出問題
    """
    prompt = f"""
    你是一位{expert}。
    
    針對主題「{topic}」，請提出 3 個重要的研究問題。
    這些問題應該：
    1. 符合你的專業領域
    2. 對讀者有價值
    3. 可以透過研究回答
    
    請直接列出問題，每個一行。
    """
    
    questions = ask_ai(prompt, role=f"你是{expert}")
    return questions.strip().split('\n')

# 測試
topic = "人工智慧"
expert = "技術專家 - 研究AI技術發展"
questions = generate_questions(topic, expert)

print(f"\n{expert} 的問題：")
for q in questions:
    print(f"  ? {q}")

## 4. 研究和收集資料

針對問題搜尋答案：

In [ ]:
def research_question(question):
    """
    研究特定問題，收集相關資訊
    """
    prompt = f"""
    問題：{question}
    
    請提供這個問題的詳細答案，包括：
    1. 核心概念解釋
    2. 重要事實或數據
    3. 實際例子
    
    用 2-3 段文字回答。
    """
    
    return ask_ai(prompt, role="你是一位知識淵博的研究員")

# 測試
question = "AI 如何學習和改進？"
answer = research_question(question)
print(f"問題：{question}")
print(f"\n研究結果：\n{answer}")

## 5. 組合成完整的研究智能體

In [ ]:
class ResearchAgent:
    """研究智能體：負責收集和整理資訊"""
    
    def __init__(self):
        self.research_data = {}  # 儲存研究資料
    
    def research_topic(self, topic):
        """
        完整的研究流程
        """
        print(f"🔍 開始研究主題：{topic}\n")
        
        # 步驟 1：創建專家
        print("步驟 1：識別專家角色")
        experts = create_expert(topic)
        for expert in experts:
            print(f"  ✓ {expert}")
        
        # 步驟 2：收集問題
        print("\n步驟 2：產生研究問題")
        all_questions = []
        for expert in experts[:2]:  # 簡化：只用前兩個專家
            questions = generate_questions(topic, expert)
            all_questions.extend(questions[:2])  # 每個專家取 2 個問題
            print(f"  ✓ {expert.split('-')[0]} 提出了 {len(questions[:2])} 個問題")
        
        # 步驟 3：研究問題
        print("\n步驟 3：研究和收集資料")
        research_results = {}
        for i, question in enumerate(all_questions[:3], 1):  # 簡化：只研究 3 個問題
            print(f"  研究問題 {i}/{len(all_questions[:3])}...")
            answer = research_question(question)
            research_results[question] = answer
        
        # 儲存結果
        self.research_data = {
            "topic": topic,
            "experts": experts,
            "questions": all_questions[:3],
            "research": research_results
        }
        
        print("\n✅ 研究完成！")
        return self.research_data
    
    def get_summary(self):
        """取得研究摘要"""
        if not self.research_data:
            return "尚未進行研究"
        
        summary = f"主題：{self.research_data['topic']}\n\n"
        summary += "研究重點：\n"
        
        for q, a in self.research_data['research'].items():
            # 只取答案的前 100 個字
            brief = a[:100] + "..." if len(a) > 100 else a
            summary += f"\n問：{q}\n答：{brief}\n"
        
        return summary

## 6. 實際使用研究智能體

In [ ]:
# 創建研究智能體
researcher = ResearchAgent()

# 研究一個主題
topic = "永續發展"
research_data = researcher.research_topic(topic)

In [ ]:
# 查看研究摘要
print("\n" + "="*50)
print("研究摘要")
print("="*50)
print(researcher.get_summary())

## 練習題

試著修改程式碼，完成以下任務：

1. **改變專家數量**：讓系統產生 5 個專家而不是 3 個
2. **調整問題深度**：讓每個專家提出 5 個問題
3. **新增主題**：試試研究「區塊鏈」或「元宇宙」

提示：只需要修改 `create_expert()` 和 `generate_questions()` 函數中的數字！

In [ ]:
# 你的練習空間
# 試著研究一個新主題



## 總結

你已經學會了：
- 創建不同的專家角色
- 從多角度產生問題
- 研究和收集資料
- 整合成完整的研究智能體

下一步：建立寫作智能體，把研究結果變成文章！